### Importação das bibliotecas

In [4]:
import pandas as pd
#import matplotlib.pyplot as plt
import requests
import csv
from io import StringIO
import ftfy

### Importação dos dados

In [5]:
url = requests.get('https://raw.githubusercontent.com/Ada-Empregabilidade/adahack-2024-dados/main/base_dados/base_de_dados.csv').content.decode('utf-8')
linhas = url.split('\n')
lista_linhas = [ftfy.fix_text(linha) for linha in linhas]
data = StringIO('\n'.join(lista_linhas))
df = pd.read_csv(data)

### Checagem dos dados importados

In [6]:
df.head(10)

,id,id.1,Nome,Genero,Idade,Raça,Endereço,Formação,Estado,Tempo de casa,Departamento,Senioridade
0,1,1,Eleonora Arilda Penedo Gomes de Padilha,Fem,34.0,pardo,"9155 Harold Oval\nSellersside, FL 21337",Ensino Médio,Santa Catarina,12.0,Compras,Analista Pleno
1,2,2,Elisângela Gabrielle de Osório,Fem,26.0,pardo,"941 Martin Manor\nLake Isaiahtown, FM 43797",Ensino Médio,Pará,6.0,Contabilidade,Analista Júnior
2,3,3,José Túlio de Cabral,Masc,35.0,pardo,"110 Davis Ridges\nMejiaville, LA 17095",Ensino Médio,Santa Catarina,5.0,Vendas,Analista Pleno
3,4,4,Ezequiel Edivaldo de Medeiros Sonao,Masc,24.0,pardo,"48010 Wilson Glen Apt. 749\nSmithborough, NV 0...",Ensino Superior,Tocantins,4.0,Administrativo,Gerente
4,5,5,Fagner Josiel dos Santos,Masc,21.0,pardo,"8666 Ramos Ports Apt. 070\nSandraport, MN 33570",Ensino Superior,Ceará,5.0,Recursos Humanos,Analista Júnior
5,6,6,Magali Luzimara da Silva,Fem,49.0,pardo,"073 Padilla Dam\nPort Joshualand, MA 65043",Ensino Superior,São Paulo,3.0,Contabilidade,Analista Pleno
6,7,7,Victor Plínio Shufrouze,Masc,29.0,pardo,"112 Rose Corners\nStanleyberg, IA 13588",Pós graduação,Ceará,9.0,Compras,Analista Sênior
7,8,8,Laércio Rossi,Masc,35.0,pardo,"508 Sean Road Apt. 754\nWest Thomaschester, AL...",Ensino Superior,Rondônia,8.0,Operações,Analista Júnior
8,9,9,Francisco Ângelo de Toledo Seixas,Masc,40.0,pardo,"9762 Goodwin Plain Apt. 560\nSchroedershire, P...",Mestrado,Amapá,1.0,Administrativo,Analista Pleno
9,10,10,Fabrício Rangel de Gonçalves dos Santos,Masc,33.0,pardo,"4979 Dana Station Apt. 126\nMichellefurt, HI 2...",Ensino Superior,Minas Gerais,3.0,Desenvolvimento de Produtos,Analista Pleno


In [7]:
df.shape

(10000, 12)

In [8]:
# Listar colunas
df.columns

Index(['id', 'id.1', 'Nome', 'Genero', 'Idade', 'Raça', 'Endereço', 'Formação',
       'Estado', 'Tempo de casa', 'Departamento', 'Senioridade'],
      dtype='object')

In [9]:
numeric_columns = df.select_dtypes(include='number')
numeric_columns.describe()

,id,id.1,Idade,Tempo de casa
count,10000.00000,10000.00000,9944.000000,9800.000000
mean,5000.50000,5000.50000,30.827836,5.490510
std,2886.89568,2886.89568,8.738915,2.963759
min,1.00000,1.00000,5.000000,-6.000000
25%,2500.75000,2500.75000,24.000000,3.000000
50%,5000.50000,5000.50000,30.000000,5.000000
75%,7500.25000,7500.25000,37.000000,8.000000
max,10000.00000,10000.00000,74.000000,17.000000


In [10]:
non_numeric_columns = df.select_dtypes(exclude='number')
non_numeric_columns.describe()

,Nome,Genero,Raça,Endereço,Formação,Estado,Departamento,Senioridade
count,10000,10000,10000,10000,9956,10000,10000,10000
unique,9961,2,5,10000,5,27,9,7
top,Francisco da Silva,Masc,pardo,"9155 Harold Oval\nSellersside, FL 21337",Ensino Superior,São Paulo,Administrativo,Analista Júnior
freq,2,7500,4700,1,5972,1000,1112,3000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             10000 non-null  int64  
 1   id.1           10000 non-null  int64  
 2   Nome           10000 non-null  object 
 3   Genero         10000 non-null  object 
 4   Idade          9944 non-null   float64
 5   Raça           10000 non-null  object 
 6   Endereço       10000 non-null  object 
 7   Formação       9956 non-null   object 
 8   Estado         10000 non-null  object 
 9   Tempo de casa  9800 non-null   float64
 10  Departamento   10000 non-null  object 
 11  Senioridade    10000 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 937.6+ KB


Não é necessário alterar os tipos de dados

Limpando os dados

In [12]:
#remover duplicatas
df = df.drop_duplicates()

In [13]:
#Alterar valores irrelevantes
df.replace(old_value, new_value, inplace=True)

NameError: name 'old_value' is not defined

In [ ]:
#remover observações irrelevantes
df = df.loc[df['Id.1'] == 'specific_value']

In [ ]:
#Remover colunas desnecessárias
df = dataset.drop(columns=['id', 'id.1', 'Nome'])

Lidando com dados faltantes

In [ ]:
#Visulalisar resultados
df.isna().sum()

In [ ]:
#Deletar dados faltantes
df = df.dropna()

In [ ]:
df.iloc[0, df.columns.get_loc('Tempo de casa')] = -6

Outliers

In [ ]:
#Boxplot
plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
dataset.boxplot()
plt.title('Boxplot for each column')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

In [ ]:
#Gráfico de dispersão

# Target column for scatter plots
target_column = 'target_column_name'

# Get numeric columns (excluding the target column)
numeric_columns = dataset.select_dtypes(include=['number']).columns.tolist()
numeric_columns.remove(target_column)

# Create scatter plots for each numeric column against the target column
for col in numeric_columns:
    plt.figure(figsize=(8, 6))
    plt.scatter(dataset[col], dataset[target_column])
    plt.title(f'Scatter plot of {col} against {target_column}')
    plt.xlabel(col)
    plt.ylabel(target_column)
    plt.grid(True)
    plt.show()

In [ ]:
#Histograma

# Get numeric columns
numeric_columns = dataset.select_dtypes(include=['number']).columns

# Plot histograms for each numeric column
for col in numeric_columns:
    plt.figure(figsize=(8, 6))
    plt.hist(dataset[col], bins=30)  # Adjust the number of bins as needed
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()